In [26]:
from sklearn import metrics
!pip install scipy numpy matplotlib pandas sklearn tabulate seaborn folium geopy geopandas requests> /dev/null
%matplotlib notebook

System nie moľe odnale«† okre�lonej �cieľki.


In [5]:
# import importlib
# import sys
# importlib.reload(sys.modules['statistics.company'])
# Load libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import pyplot
from sklearn.model_selection import train_test_split, learning_curve, permutation_test_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import normalize
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, plot_roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score, silhouette_score
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tabulate import tabulate
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import os
import sys
sys.path.insert(0, os.path.abspath('./module-dir'))

In [6]:
from dataset_provider.provider import get_dataset
from dataset_provider.config import create_global_config
cfg = create_global_config()
df = get_dataset(cfg)

Taking data from local file...
Found 304 job ads without salary range or currency
(1249, 75)
        salary_from     salary_to
count   1249.000000   1249.000000
mean   11148.760608  15884.851081
std     4540.909690   5114.312359
min     1250.000000   1750.000000
25%     8000.000000  12100.000000
50%    10000.000000  16000.000000
75%    14000.000000  19000.000000
max    44000.000000  50000.000000
Found 8 countries and 4 currencies!
Dropping foreign countries and translating currencies...
Unique countries: 1, currencies: 1, observations: 1230


C:\Users\Pacia\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


# Statistics

### Plot company sizes and salary ranges

In [7]:
from statistics.company import plot_stats
plot_stats(df)

Found 75 jobs with salary over 80000, which won't be taken into account on plots below.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
from statistics.percentage import count_percentage
count_percentage(df)

,Count,Percentage
javascript,222,18.0%
java,169,13.7%
php,97,7.9%
net,97,7.9%
devops,96,7.8%
other,95,7.7%
mobile,86,7.0%
testing,76,6.2%
python,69,5.6%
data,52,4.2%


<IPython.core.display.Javascript object>

,Count,Percentage
mid,730,59.3%
senior,448,36.4%
junior,52,4.2%


<IPython.core.display.Javascript object>

,Count,Percentage
b2b,903,73.4%
permanent,308,25.0%
mandate_contract,19,1.5%


<IPython.core.display.Javascript object>

---------------------------------------------------------------------------

# Heat maps

In [9]:
from heat_maps.density_map import density_heat_map
density_heat_map(df)

<h2>Get province data </h2>

In [10]:
from heat_maps.salary_map import show_map
show_map(df)

D:\STUDIA\Magisterka\1 semestr\Uczenie maszynowe\jupyter\module-dir\heat_maps\density_map.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  province_data[province_id_column_name].astype(int)


---------------------------------------------------------------------------

# Classification

In [30]:
cl_df = df.copy()
for col in ["city", "Unnamed: 0", "title", "company_size", "country_code", "marker_icon", "company_name",
            "latitude", "longitude", "salary_currency", "published_at", "remote_interview", "id",
            "Vert.x", "skills", "-", 'salary_from', 'salary_to', 'remote']:
                cl_df=cl_df.drop(col, axis=1)
print(cl_df.columns)
cols_for_encoding = [c for c in cl_df.select_dtypes(include=['object']).copy().columns if c != "experience_level"]
print(cols_for_encoding)
cl_df = pd.get_dummies(cl_df, columns=cols_for_encoding)

Index(['experience_level', 'employment_type', '.NET', 'Design', 'Network',
       'REST API', 'Embeded', 'Cloud', 'Database', 'Android', 'IT',
       'Soft Skills', 'Scrum master', 'Mobile', 'Common', 'JavaScript',
       'DevOps', 'Software engineering', 'Testing', 'Automation',
       'Shell Scripting', 'Backend', 'Data Science', 'Blockchain', 'C++',
       'Client Service', 'Front-end', 'Civil Engineering', 'Developer', 'iOS',
       'SQL', 'Python', 'PHP', 'Erlang', 'Scala', 'Git', 'Games', 'Golang',
       'Google', 'Java', 'Web', 'Consultant', 'Project Manager', 'Analitics',
       'CRM', 'Perl', 'R', 'Ruby', 'Rust', 'SAP', 'CSS', 'XML', 'Salesforce',
       'API', 'Data', 'Excel'],
      dtype='object')
['employment_type']


In [31]:
print(len(cl_df.columns))

58


In [32]:
def move_col_to_end(df, col):
    cols_at_end = [col]
    return df[[c for c in df if c not in cols_at_end] 
            + [c for c in cols_at_end if c in df]]

cl_df = move_col_to_end(cl_df, "experience_level")
array = cl_df.values
x = array[:,0:len(cl_df.columns)-1]
y = array[:,len(cl_df.columns)-1]
x = normalize(x)
le = LabelEncoder()
y = le.fit_transform(y)
#na podstawie x i y otrzymujemy tablice testowe i wynikowe
x_train, x_validation, y_train, y_validation = train_test_split(x,y, test_size=cfg.test_size, random_state=1)

In [14]:
models = []
models.extend([
    ('KNN', KNeighborsClassifier(), 0),
    ('CART', DecisionTreeClassifier(), 1),
    ('NB', GaussianNB(), 2),
    ('SVM', SVC(gamma='auto'), 3),
    ('MLP', MLPClassifier(alpha=1e-5, hidden_layer_sizes=(50,10), max_iter=5000), 4)
    ])
def plot_show():
    pyplot.draw()
    pyplot.pause(0.1)

def get_specificity(y_validate, y_predicted):
    cnf_matrix = confusion_matrix(y_validate, y_predicted)
    
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)
    
    FP = FP.astype(float)
    TN = TN.astype(float)
        
    return np.mean(TN/(TN+FP))
    
def get_learning_curve(classification_model, training_set_enlarging_step=10):
    train_sizes = np.linspace(0.1, 1, training_set_enlarging_step)
    train_sizes, train_scores, validation_scores = learning_curve(
        estimator = classification_model,
        X = x,
        y = y, 
        train_sizes = train_sizes,
        cv = 5,
        scoring = 'accuracy')
    return train_sizes, train_scores, validation_scores

def plot_learning_curve(model,name):
    train_sizes, train_scores, test_scores = \
        get_learning_curve(model)

    plt.style.use('seaborn')
    plt.plot(train_sizes, -train_scores.mean(axis = 1), color= 'red', label = 'Training error')
    plt.plot(train_sizes, -test_scores.mean(axis = 1), color= 'navy',label = 'Validation error')
    plt.ylabel('Accuracy', fontsize = 14)
    plt.xlabel('Training set size', fontsize = 14)
    plt.title('Learning curves for a %s' % name, fontsize = 18, y = 1.03)
    plt.legend()
    plt.show()
    
def compare_algorithms(results, names):
    fig = pyplot.figure()
    fig.suptitle("Algorithm Comparison")
    ax = fig.add_subplot(1,1,1)
    ax.set_title("Algorithm Comparison")
    ax.boxplot(results, labels=names)
    plot_show()

def print_scores(cv_results, predictions):
    print('\nMean %f' % cv_results.mean())
    print('STD %f' % cv_results.std())
    print('\nConfusion matrix:')
    print(confusion_matrix(y_validation, predictions))
    print('\nAccuracy %f' % accuracy_score(y_validation, predictions))
    print('Precision %f' % precision_score(y_validation, predictions, average = 'weighted'))
    print('Recall %f' % recall_score(y_validation, predictions, average = 'weighted'))
    print('Specificity %f' % get_specificity(y_validation, predictions))
    print('\nClassification report:')
    print(classification_report(y_validation, predictions))

def plot_roc_curves():
    fig = pyplot.figure()
    ax = plt.gca()
    for name, model, subplot_row in models:
        rfc_disp = plot_roc_curve(model, x_validation, 
                                  y_validation, ax=ax, alpha=0.8)
    plt.show()

def accuracySignificancy(model, x_train, y_train, cv):
    fig = pyplot.figure()
    fig.suptitle("Estimating accuracy score's statistical significancy")
    ax = fig.add_subplot(1,1,1)
    n_classes = np.unique(y_train).size
    score, permutation_scores, pvalue = permutation_test_score(model, x_train, y_train, scoring="accuracy", cv=cv, n_permutations=100)
    print("Classification score %s (pvalue : %s)" % (score, pvalue))
    # View histogram of permutation scores
    ax.hist(permutation_scores, 20, label='Permutation scores',
             edgecolor='black')
    ylim = plt.ylim()
    ax.set_xlabel('Score')
    plt.show()

def evaluate_per_dataset():
    results = []
    names = []
    
    for name, model, subplot_row in models:
            print(f"---------------------------\nRunning classification for: {name}")
            kfold = StratifiedKFold(n_splits=4, random_state=1, shuffle=True)
            cv_results = cross_val_score(model, x_train, y_train, cv=kfold, scoring='accuracy')
            accuracySignificancy(model, x_train, y_train, kfold)
            results.append(cv_results)
            names.append(name)

            # Make predictions on validation dataset
            model.fit(x_train, y_train)
            predictions = model.predict(x_validation)
            
            print_scores(cv_results, predictions)
            plot_learning_curve(model,name)

    # Compare Algorithms - ROC etc
    compare_algorithms(results, names)

In [33]:
evaluate_per_dataset()

TypeError: cannot unpack non-iterable HuberRegressor object

# Regression

In [16]:
print(cl_df.head())

   remote  salary_from  salary_to  .NET  Design  Network  REST API  Embeded  \
0   False      12000.0    15000.0     0       0        0         0        0   
1   False      12000.0    18000.0     0       0        0         5        0   
2   False      10000.0    16000.0     0       0        0         0        0   
5    True      12000.0    18000.0     0       0        0         0        0   
6   False       3500.0     5500.0     0       0        0         0        0   

   Cloud  Database  ...  CSS  XML  Salesforce  API  Data  Excel  \
0      0         0  ...    0    0           0    0     0      0   
1      0         0  ...    0    0           0    0     0      0   
2      0         0  ...    0    0           0    0     0      0   
5      0         0  ...    0    0           0    0     0      0   
6      0         0  ...    0    0           0    0     0      0   

   employment_type_b2b  employment_type_mandate_contract  \
0                    1                                 0   
1 

In [17]:
cl_df = df.copy()
for col in ["salary_to", "city", "Unnamed: 0", "title", "company_size", "country_code", "marker_icon", "company_name",
            "latitude", "longitude", "salary_currency", "published_at", "remote_interview", "id",
            "Vert.x", "skills", "-"]:
                cl_df=cl_df.drop(col, axis=1)

In [18]:
cols_for_encoding = [c for c in cl_df.select_dtypes(include=['object', 'boolean']).copy().columns]
cl_df = pd.get_dummies(cl_df, columns=cols_for_encoding)
cl_df = move_col_to_end(cl_df, "salary_from")
array = cl_df.values
x = array[:, 0:len(cl_df.columns)-1]
y = array[:, len(cl_df.columns)-1]
x = normalize(x)
print(x[0])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.91766294 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.22941573 0.         0.         0.         0.22941573 0.22941573
 0.         0.        ]


In [19]:
print(x[0])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.91766294 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.22941573 0.         0.         0.         0.22941573 0.22941573
 0.         0.        ]


In [20]:
x_train, x_validation, y_train, y_validation = train_test_split(x,y, test_size=cfg.test_size, random_state=1)

In [21]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components = 56)
# x = pca.fit_transform(x)
# print(len(x[0]))
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()  # doctest: +SKIP
scaler.fit(x_train)  # doctest: +SKIP
x_train = scaler.transform(x_train)  # doctest: +SKIP
x_validation = scaler.transform(x_validation)  # doctest: +SKIP

In [22]:
cl_df.columns

Index(['.NET', 'Design', 'Network', 'REST API', 'Embeded', 'Cloud', 'Database',
       'Android', 'IT', 'Soft Skills', 'Scrum master', 'Mobile', 'Common',
       'JavaScript', 'DevOps', 'Software engineering', 'Testing', 'Automation',
       'Shell Scripting', 'Backend', 'Data Science', 'Blockchain', 'C++',
       'Client Service', 'Front-end', 'Civil Engineering', 'Developer', 'iOS',
       'SQL', 'Python', 'PHP', 'Erlang', 'Scala', 'Git', 'Games', 'Golang',
       'Google', 'Java', 'Web', 'Consultant', 'Project Manager', 'Analitics',
       'CRM', 'Perl', 'R', 'Ruby', 'Rust', 'SAP', 'CSS', 'XML', 'Salesforce',
       'API', 'Data', 'Excel', 'remote_False', 'remote_True',
       'experience_level_junior', 'experience_level_mid',
       'experience_level_senior', 'employment_type_b2b',
       'employment_type_mandate_contract', 'employment_type_permanent',
       'salary_from'],
      dtype='object')

In [23]:
from sklearn.linear_model import LinearRegression, LogisticRegression, BayesianRidge, Lasso, ElasticNet, Lars, OrthogonalMatchingPursuit, SGDRegressor, ARDRegression
from sklearn.linear_model import MultiTaskLasso, MultiTaskLassoCV, HuberRegressor, TheilSenRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error

def plot_data(axes, title, x_train, y_train, x_test, y_test):
    axes.set_title(title)
    axes.scatter(x_train[:,0], y_train, color="green"),
    axes.plot(x_test, y_test, color="red", linewidth=1)
    
models = [
#     LinearRegression(),
    HuberRegressor(alpha=0.00001, epsilon=1.6, max_iter=5000),
    BayesianRidge(),
#     SGDRegressor(),
#     ARDRegression(),
#     TheilSenRegressor(),
#     MultiTaskLasso(),
#     MultiTaskLassoCV(),
#     OrthogonalMatchingPursuit(),
#     Lars(),
#     MLPRegressor(hidden_layer_sizes=(9), solver='lbfgs', tol=1e-6, learning_rate='adaptive', max_iter=3000, n_iter_no_change=100),
#     MLPRegressor(hidden_layer_sizes=(9), alpha=1e-8, solver='lbfgs', tol=1e-6, learning_rate='adaptive', max_iter=3000, n_iter_no_change=100),
#     MLPRegressor(hidden_layer_sizes=(9), alpha=1, solver='lbfgs', tol=1e-6, learning_rate='adaptive', max_iter=3000, n_iter_no_change=100),
#     MLPRegressor(hidden_layer_sizes=(16), solver='sgd', learning_rate='adaptive', max_iter=500, n_iter_no_change=100),
#     SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1),
#     SVR(kernel='poly', C=100, gamma='scale', degree=3, epsilon=.1, coef0=1),
#     SVR(kernel='linear', C=100, gamma='scale'),
#     SVR(kernel='rbf'),
#     SVR(kernel='poly', degree=60, gamma='scale'),
#     SVR(kernel='linear', gamma='scale'),    
#     SVR(kernel='sigmoid', gamma='scale'),
#     SVR(kernel='precomputed', gamma='auto'),
    Lasso(alpha=0.1, max_iter=1000_00, random_state=True),
    ElasticNet(alpha=0.01, max_iter=1000_00, random_state=True)]

# f, axarr = plt.subplots(len(models), sharex=True, sharey=True,figsize=(12,12))
for y, model in enumerate(models):
        y_test = model.fit(x_train, y_train).predict(x_validation)
#         plot_data(axarr[y], type(model).__name__, x_train, y_train, x_validation, y_test)
        print(f"{type(model).__name__} MAE: {mean_absolute_error(y_validation, y_test, )} | MSE: {mean_squared_error(y_validation, y_test, )}")

HuberRegressor MAE: 2750.816973155815 | MSE: 11739924.753626151
BayesianRidge MAE: 2813.6850494994146 | MSE: 12018607.014733346
Lasso MAE: 2761.1972806208155 | MSE: 11777501.631878218
ElasticNet MAE: 2767.055819759564 | MSE: 11803452.700619938


In [24]:
for i, y in enumerate(y_validation):
    print(f"predicted: {y_test[i]} original: {y_validation[i]}")

predicted: 9544.520084775852 original: 16800.0
predicted: 13296.168823952297 original: 10000.0
predicted: 4964.631782467768 original: 3500.0
predicted: 12834.262031942093 original: 8000.0
predicted: 10465.258994447468 original: 8000.0
predicted: 14285.323477372214 original: 15000.0
predicted: 14702.026785657836 original: 16000.0
predicted: 7418.163122155673 original: 6000.0
predicted: 10037.074308022677 original: 8000.0
predicted: 7653.3602067499805 original: 3500.0
predicted: 16832.26213561563 original: 16000.0
predicted: 14883.950350856128 original: 10000.0
predicted: 12807.143020079153 original: 14000.0
predicted: 10598.256223628188 original: 9000.0
predicted: 11777.067916168588 original: 13000.0
predicted: 8869.628527439156 original: 8000.0
predicted: 11839.589509345828 original: 8000.0
predicted: 10584.384953089115 original: 6000.0
predicted: 11339.057825309887 original: 14000.0
predicted: 10966.485818016603 original: 6000.0
predicted: 8236.152291719605 original: 7000.0
predicted:

In [25]:
# print(__doc__)


# # Code source: Gaël Varoquaux
# # Modified for documentation by Jaques Grobler
# # License: BSD 3 clause

# import numpy as np
# import matplotlib.pyplot as plt
# import pandas as pd

# from sklearn import datasets
# from sklearn.decomposition import PCA
# from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import GridSearchCV

# pca = PCA()
# bayesian = BayesianRidge() #(max_iter=10000, tol=0.1)
# pipe = Pipeline(steps=[('pca', pca), ('bayesian', bayesian)])
# param_grid = {
#     'pca__n_components': [5, 15, 30, 45, 64]
# }
# search = GridSearchCV(pipe, param_grid, n_jobs=-1)
# search.fit(x, y)
# print("Best parameter (CV score=%0.3f):" % search.best_score_)
# print(search.best_params_)

# # Plot the PCA spectrum
# pca.fit(x)

# fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True, figsize=(6, 6))
# ax0.plot(np.arange(1, pca.n_components_ + 1),
#          pca.explained_variance_ratio_, '+', linewidth=2)
# ax0.set_ylabel('PCA explained variance ratio')

# ax0.axvline(search.best_estimator_.named_steps['pca'].n_components,
#             linestyle=':', label='n_components chosen')
# ax0.legend(prop=dict(size=12))

# # For each number of components, find the best classifier results
# results = pd.DataFrame(search.cv_results_)
# components_col = 'param_pca__n_components'
# best_clfs = results.groupby(components_col).apply(
#     lambda g: g.nlargest(1, 'mean_test_score'))

# best_clfs.plot(x=components_col, y='mean_test_score', yerr='std_test_score',
#                legend=False, ax=ax1)
# ax1.set_xlabel('n_components')

# plt.xlim(-1, 70)

# plt.tight_layout()
# plt.show()